<a href="https://colab.research.google.com/github/bforoura/Transformers/blob/main/nlp_transformers_ch8_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title A Positional Encoding Example

# Gensim is a free open-source Python library for representing documents as semantic vectors
!pip install gensim==3.8.3


# An open-source machine learning library, a scientific computing framework, and a 
# script language based on the Lua programming language.
import torch

# Library is a suite that contains libraries and programs for statistical language processing
import nltk

# Punktis a Sentence Tokenizer that divides a text into a list of sentences by using 
# an unsupervised algorithm to build a model for abbreviation words, collocations, and 
# words that start sentences.
nltk.download('punkt')



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.2 MB 1.9 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#@title Necessary imports

import math
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize 
import gensim 
from gensim.models import Word2Vec 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings(action = 'ignore') 



In [8]:
#@title Upload the data file and parse it

dprint=1 # prints outputs if set to 1, default=0

#‘text.txt’ file 
!curl -L https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/main/Chapter01/text.txt --output "test.txt"

sample = open("test.txt", "r") 
s = sample.read() 

# processing escape characters 
f = s.replace("\n", " ") 

data = [] 

# sentence parsing 
for i in sent_tokenize(f): 
	temp = [] 
	# tokenize the sentence into words 
	for j in word_tokenize(i): 
		temp.append(j.lower()) 
	data.append(temp) 





  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24151  100 24151    0     0   436k      0 --:--:-- --:--:-- --:--:--  444k


In [9]:
#@title Create the Skip Gram models

#model2 = gensim.models.Word2Vec(data, min_count = 1, size = 512,window = 5, sg = 1) 
#model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 512,window = 5, sg = 1)

# 1-The 2-black 3-cat 4-sat 5-on 6-the 7-couch 8-and 9-the 10-brown 11-dog 12-slept 13-on 14-the 15-rug.
word1='black'
word2='brown'

pos1=2
pos2=10

a=model2[word1]
b=model2[word2]

if(dprint==1):
        print(a)


[ 0.03598683 -0.00565577 -0.10151643  0.03018838  0.01860851 -0.01856142
  0.00210228 -0.03402211  0.06821775  0.05761792  0.07001607 -0.01343211
  0.03389274 -0.07457796 -0.04713601 -0.01342907 -0.06210098 -0.01079821
  0.02137231 -0.07676471 -0.04403317  0.01638177 -0.07369281  0.07799415
 -0.01275421  0.04845377  0.02827115 -0.01043866  0.1370989   0.0510955
 -0.03608342 -0.05271146  0.02822991 -0.03588377  0.01670334 -0.03939424
 -0.00254495  0.078578   -0.04785986  0.01141417 -0.04350094 -0.07098354
 -0.10861259 -0.1241564  -0.00638527  0.01881578 -0.00819512 -0.04847056
  0.07769629 -0.07984544 -0.13270716  0.00524202 -0.06403963 -0.04899118
  0.0689587   0.00789011 -0.02186745  0.06444667  0.13739644  0.05977032
 -0.01969501  0.01093789  0.02829748 -0.05757109 -0.02741261 -0.05213709
 -0.06093855 -0.00292054  0.03714421  0.00382517  0.01853587 -0.06361522
 -0.01774343 -0.00147828  0.005125   -0.04781511 -0.00887499 -0.00228059
  0.05194479  0.12663615 -0.05377701  0.15178393  0.